In [1]:
import numpy as np
from czml3 import CZML_VERSION, Document, Packet
from czml3.properties import (
    Color,
    Material,
    Polygon,
    PositionList,
    SolidColorMaterial,
)

import czml3_ext
from czml3_ext import packets
from czml3_ext.colours import (
    RGBA_black,
    RGBA_orange,
    create_palette,
)
from czml3_ext.helpers import get_contours, get_coverage_amount

In [ ]:
print(f"Using czml3-ext version: {czml3_ext.__version__}")

## Coverage
Amount of coverage overlap

In [3]:
arr, origin_x1, origin_y1, delta_x1, delta_y1 = get_coverage_amount(
    ["data_cut.tif", "data.tif"],
    5,
)

In [4]:
rgba = create_palette(
    [RGBA_black, RGBA_black, RGBA_orange], np.max(arr) + 1
)  # repeating the first colour twice because we don't want to show a zero coverage

In [5]:
overlaps = []
for num_covers in range(1, np.max(arr) + 1):
    coverage_overlap, _ = get_contours(
        arr == num_covers,
        deg_origin_x=origin_x1,
        deg_size_x=delta_x1,
        deg_origin_y=origin_y1,
        deg_size_y=delta_y1,
    )
    overlaps.extend(
        packets.coverage(
            coverage_overlap,
            name=f"Coverage of {num_covers}",
            polygon=Polygon(
                positions=PositionList(cartographicDegrees=[0, 0, 0]),
                material=Material(
                    solidColor=SolidColorMaterial(color=Color(rgba=rgba[num_covers]))
                ),
            ),
        )
    )

## Export
Export to file as you would any other `czml3.Document`.

In [6]:
doc = Document(
    packets=([Packet(name="example", id="document", version=CZML_VERSION)] + overlaps)
)

In [7]:
with open("example.czml", "w") as f:
    f.write(doc.dumps())